In [1]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [9]:
import pandas as pd
import numpy as np
parent_file=pd.read_pickle(r'master4.pkl')
for i in ['L1',"L2","L3","L4","L5"]:
    parent_file[i]=parent_file[i].apply(str)
parent_file.head()

,index,L1,L2,L3,L4,L5,Alternate_words
0,1,Leukemia,,,,,"{'L1': ['Leukemia', 'hematologic malignancies/..."
1,2,Leukemia,Acute Myelogenous Leukemia,,,,"{'L1': ['Leukemia', 'hematologic malignancies/..."
2,3,Leukemia,Acute Myelogenous Leukemia,AML with minimal differentiation (M0),,,"{'L1': ['Leukemia', 'hematologic malignancies/..."
3,4,Leukemia,Acute Myelogenous Leukemia,AML without maturation (M1),,,"{'L1': ['Leukemia', 'hematologic malignancies/..."
4,5,Leukemia,Acute Myelogenous Leukemia,AML with maturation (M2),,,"{'L1': ['Leukemia', 'hematologic malignancies/..."


In [10]:
parent_file.columns=['index', 'L1', 'L2', 'L3', 'L4', 'L5', 'master']
parent_file.head()

,index,L1,L2,L3,L4,L5,master
0,1,Leukemia,,,,,"{'L1': ['Leukemia', 'hematologic malignancies/..."
1,2,Leukemia,Acute Myelogenous Leukemia,,,,"{'L1': ['Leukemia', 'hematologic malignancies/..."
2,3,Leukemia,Acute Myelogenous Leukemia,AML with minimal differentiation (M0),,,"{'L1': ['Leukemia', 'hematologic malignancies/..."
3,4,Leukemia,Acute Myelogenous Leukemia,AML without maturation (M1),,,"{'L1': ['Leukemia', 'hematologic malignancies/..."
4,5,Leukemia,Acute Myelogenous Leukemia,AML with maturation (M2),,,"{'L1': ['Leukemia', 'hematologic malignancies/..."


In [11]:
# Reading an Excel file into a DataFrame from the specified file path
df_palm = pd.read_excel(r"C:\Users\basem\OneDrive\Desktop\shakaib\Tumor module\TTL pkg\TTL_updated_Master_File_Dec_2024\Reference_files\Master_files\parent2_ttl_sequenced_Dec_24.xlsx")

# Displaying the column names of the loaded DataFrame
# Useful for verifying the structure and understanding the available data fields
df_palm.columns

Index(['NCTId', 'concatenated_text', 'output_PALM2', 'output_PALM2_updated',
       'NERs', 'basem_transformed_values', 'prod_over_palm', 'ttl_tags', 'met',
       'anat', 'other_tags', 'Tumor_Type_Missed_Tags'],
      dtype='object')

In [17]:
# Function to process and filter TTL entities from a given list of tuples
def fn_updated_TTL(txt: list) -> pd.DataFrame:
    # Check if the input list is not empty
    if len(txt) > 0:  # Extract Basem unique NERs for TTL1 to TTL4
        # Filter the input list for tuples with specific TTL entity types
        txt: list[tuple[str, str]] = [i for i in txt if i[1] in ['TTL_NE', 'TTL1', 'TTL2', 'TTL3', 'TTL4']]
        
        # Remove duplicate entries based on both elements of the tuple
        txt1: list[tuple[str, str]] = []
        for i in txt:
            if len(txt1) == 0:
                txt1.append(i)
            else:
                for j in txt1:
                    if j[0] == i[0] and j[1] == i[1]:  # Skip duplicates
                        break
                else:
                    txt1.append(i)
        
        # Copy the parent DataFrame for processing
        parent_file1 = parent_file.copy()
        df_22: list = []  # List to store matching rows
        not_found: list = []  # List to store unmatched TTL entities
        found: list = []  # List to track found TTL entities

        # Iterate through the filtered TTL list
        for st in txt1:
            counter = 0  # Counter to track matches for the current TTL entity
            for idx, row in parent_file1.iterrows():
                l = row['master']
                if isinstance(l, dict):  # Ensure the 'master' column contains a dictionary
                    for k in ['L4', 'L3', 'L2', 'L1']:  # Search in specific levels of the dictionary
                        l1 = l.get(k)
                        if isinstance(l1, list):  # Check if the value is a list
                            for j in l1:
                                # Match TTL entity (case-insensitive comparison)
                                if str(j).lower().strip() == str(st[0]).lower().strip():
                                    df_22.append(row)  # Append matching row to results
                                    found.append(st[0])  # Mark the entity as found
                                    counter += 1
                                    break
            
            # If no matches found, add the TTL entity to the not_found list
            if counter == 0:
                not_found.append(st[0])
        
        # Attempt to create a DataFrame from matched rows and deduplicate results
        try:
            df_22 = pd.concat(df_22, axis=1).T  # Combine rows into a single DataFrame
            df_22 = pd.DataFrame(df_22).drop_duplicates(subset=['index'])  # Remove duplicates
            
            # Convert found TTL entities to a unique list
            found = list(set(found))
            
            # If no results, return an empty DataFrame and unmatched entities
            if len(df_22) == 0:
                return empty_df, not_found
            
            # Return the processed DataFrame and unmatched entities
            return df_22[['L1', 'L2', 'L3', 'L4', 'L5']].values.tolist(), not_found
        except Exception as e:
            # Handle any exceptions gracefully and return an empty DataFrame
            print(f"Error encountered: {e}")
            empty_df = pd.DataFrame()
            return empty_df, not_found

# Call the function with the input list 'txt'
df_22, not_found = fn_updated_TTL(txt)

# Print unmatched TTL entities for debugging or review
print(not_found)

# Display the processed DataFrame
df_22

[]


[['Breast Cancer', '', '', '', ''],
 ['Breast Cancer',
  'Invasive Ductal Cancer of the Breast (for inflammatory breast cancer please also run query for inflammatory cancer as well)',
  '',
  '',
  ''],
 ['Breast Cancer', 'inflammatory breast cancer', '', '', ''],
 ['Breast Cancer',
  'Invasive Lobular Cancer of the Breast (for inflammatory breast cancer please also run query for inflammatory cancer as well)',
  '',
  '',
  ''],
 ['Breast Cancer', 'Ductal Carcinoma In Situ (stage 0)', '', '', ''],
 ['Breast Cancer', 'Metaplastic Breast Cancer', '', '', ''],
 ['Breast Cancer', 'Pure tubular breast cancer', '', '', ''],
 ['Breast Cancer',
  'Pure mucinous breast cancer (includes mucinous ductal breast cancer)',
  '',
  '',
  ''],
 ['Breast Cancer', 'Pure cribriform breast cancer', '', '', ''],
 ['Breast Cancer',
  'Papillary carcinoma (includes papillary ductal breast cancer)',
  '',
  '',
  ''],
 ['Breast Cancer', 'Adenoid cystic breast cancer', '', '', ''],
 ['Breast Cancer', 'medullar

In [ ]:
df_palm=df_palm.drop(columns=['prodigy_over_palm_refined'])

In [15]:
%%time
# df_palm
# Apply the function to the filtered DataFrame
df_palm['prodigy_over_palm_refined'] = df_palm['prod_over_palm'].apply(extract_tumor_info)
df_palm

CPU times: total: 922 ms
Wall time: 2.04 s


,NCTId,concatenated_text,output_PALM2,output_PALM2_updated,NERs,basem_transformed_values,prod_over_palm,ttl_tags,met,anat,other_tags,Tumor_Type_Missed_Tags,prodigy_over_palm_refined
0,NCT00909506,NCT00909506\nBreast Cancer\nNone\nEfficacy and...,"{\n ""tumors"": [\n {\n ""type"": ""Breast...","{\n ""tumors"": [\n {\n ""type"": ""breast...","[('NCT00909506', 'nct'), ('Breast Cancer', 'TT...","('breast cancer TTL1 stage operable st_v', [([...","[[{'Tumor Type': 'breast cancer', 'Tag': 'TTL1...","[{'Tumor Type': 'breast cancer', 'Tag': 'TTL1'}]",[],[],[],[],"[(breast cancer, TTL1)]"
1,NCT03473587,NCT03473587\nColorectal Cancer\nNone\nPromotin...,"{\n ""tumors"": [\n {\n ""type"": ""Colore...","{\n ""tumors"": [\n {\n ""type"": ""colore...","[('NCT03473587', 'nct'), ('Colorectal Cancer',...",('colorectal cancer TTL1 includes inc colon TT...,[[]],[],[],[],[],[],[]
2,NCT01411904,NCT01411904\nLeukemia\nLeukemia\nmagnetic need...,"{\n ""tumors"": [\n {\n ""type"": ""Leukem...","{\n ""tumors"": [\n {\n ""type"": ""leukem...","[('NCT01411904', 'nct'), ('Leukemia', 'TTL1'),...","('leukemia TTL1 stage', [([('leukemia', 'TTL1'...","[[{'Tumor Type': 'leukemia', 'Tag': 'TTL1'}]]","[{'Tumor Type': 'leukemia', 'Tag': 'TTL1'}]",[],[],[],[],"[(leukemia, TTL1)]"
3,NCT01925001,"NCT01925001\nAnemia, Sickle Cell\nSickle Cell ...","{\n ""tumors"": [\n {\n ""type"": ""Sickle...","{\n ""tumors"": [\n {\n ""type"": ""sickle...","[('NCT01925001', 'nct'), ('Anemia, Sickle Cell...","('sickle cell anemia ben stage null null', [([...",[[]],[],[],[],[],[],[]
4,NCT01432145,NCT01432145\nBreast Cancer\nOvarian Cancer\nBR...,"{\n ""tumors"": [\n {\n ""type"": ""Ovaria...","{\n ""tumors"": [\n {\n ""type"": ""ovaria...","[('NCT01432145', 'nct'), ('Breast Cancer', 'TT...",('ovarian cancer TTL1 stage metastatic st_v\nb...,"[[{'Tumor Type': 'ovarian cancer', 'Tag': 'TTL...","[{'Tumor Type': 'ovarian cancer', 'Tag': 'TTL1...",[],[],[],[],"[(ovarian cancer, TTL1), (breast cancer, TTL1)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104762,NCT04767152,NCT04767152\nAcute Leukemia\nQuantitative MRI\...,"{\n ""tumors"": [\n {\n ""type"": ""Acute ...","{\n ""tumors"": [\n {\n ""type"": ""acute ...","[('NCT04767152', 'nct'), ('Acute Leukemia', 'T...",('acute lymphoblastic leukemia TTL2 stage null...,[[]],[],[],[],[],[],[]
104763,NCT04115995,NCT04115995\nReconstructive Surgical Procedure...,"{\n ""tumors"": [\n {\n ""type"": ""Breast...","{\n ""tumors"": [\n {\n ""type"": ""breast...","[('NCT04115995', 'nct'), ('Reconstructive Surg...","('breast cancer TTL1 stage null null', [([('br...","[[{'Tumor Type': 'breast cancer', 'Tag': 'TTL1...","[{'Tumor Type': 'breast cancer', 'Tag': 'TTL1'}]",[],[],[],[],"[(breast cancer, TTL1)]"
104764,NCT04412135,NCT04412135\nHematologic Cancer\nNone\nThe Poi...,"{\n ""tumors"": [\n {\n ""type"": ""Hemato...","{\n ""tumors"": [\n {\n ""type"": ""hemato...","[('NCT04412135', 'nct'), ('Hematologic Cancer'...","('hematologic cancer TTL1 stage null null', [(...",[[]],[],[],[],[],[],[]
104765,NCT05490017,NCT05490017\nParoxysmal Nocturnal Hemoglobinur...,"{\n ""tumors"": []\n}","{\n ""tumors"": []\n}","[('NCT05490017', 'nct'), ('Paroxysmal Nocturna...","('', [])",[[]],[],[],[],[],[],[]


In [18]:
%%time
# df_palm
# Apply the function to the filtered DataFrame
df_palm[['Post_Processing_results','No_post_processing_equivalency']]=df_palm[['prodigy_over_palm_refined']].apply(lambda x:fn_updated_TTL(*x),axis=1,result_type='expand')
df_palm

CPU times: total: 1h 57min 54s
Wall time: 3h 55min 13s


,NCTId,concatenated_text,output_PALM2,output_PALM2_updated,NERs,basem_transformed_values,prod_over_palm,ttl_tags,met,anat,other_tags,Tumor_Type_Missed_Tags,prodigy_over_palm_refined,Post_Processing_results,No_post_processing_equivalency
0,NCT00909506,NCT00909506\nBreast Cancer\nNone\nEfficacy and...,"{\n ""tumors"": [\n {\n ""type"": ""Breast...","{\n ""tumors"": [\n {\n ""type"": ""breast...","[('NCT00909506', 'nct'), ('Breast Cancer', 'TT...","('breast cancer TTL1 stage operable st_v', [([...","[[{'Tumor Type': 'breast cancer', 'Tag': 'TTL1...","[{'Tumor Type': 'breast cancer', 'Tag': 'TTL1'}]",[],[],[],[],"[(breast cancer, TTL1)]","[[Breast Cancer, , , , ], [Breast Cancer, Inva...",[]
1,NCT03473587,NCT03473587\nColorectal Cancer\nNone\nPromotin...,"{\n ""tumors"": [\n {\n ""type"": ""Colore...","{\n ""tumors"": [\n {\n ""type"": ""colore...","[('NCT03473587', 'nct'), ('Colorectal Cancer',...",('colorectal cancer TTL1 includes inc colon TT...,[[]],[],[],[],[],[],[],None,None
2,NCT01411904,NCT01411904\nLeukemia\nLeukemia\nmagnetic need...,"{\n ""tumors"": [\n {\n ""type"": ""Leukem...","{\n ""tumors"": [\n {\n ""type"": ""leukem...","[('NCT01411904', 'nct'), ('Leukemia', 'TTL1'),...","('leukemia TTL1 stage', [([('leukemia', 'TTL1'...","[[{'Tumor Type': 'leukemia', 'Tag': 'TTL1'}]]","[{'Tumor Type': 'leukemia', 'Tag': 'TTL1'}]",[],[],[],[],"[(leukemia, TTL1)]","[[Leukemia, , , , ], [Leukemia, Acute Myelogen...",[]
3,NCT01925001,"NCT01925001\nAnemia, Sickle Cell\nSickle Cell ...","{\n ""tumors"": [\n {\n ""type"": ""Sickle...","{\n ""tumors"": [\n {\n ""type"": ""sickle...","[('NCT01925001', 'nct'), ('Anemia, Sickle Cell...","('sickle cell anemia ben stage null null', [([...",[[]],[],[],[],[],[],[],None,None
4,NCT01432145,NCT01432145\nBreast Cancer\nOvarian Cancer\nBR...,"{\n ""tumors"": [\n {\n ""type"": ""Ovaria...","{\n ""tumors"": [\n {\n ""type"": ""ovaria...","[('NCT01432145', 'nct'), ('Breast Cancer', 'TT...",('ovarian cancer TTL1 stage metastatic st_v\nb...,"[[{'Tumor Type': 'ovarian cancer', 'Tag': 'TTL...","[{'Tumor Type': 'ovarian cancer', 'Tag': 'TTL1...",[],[],[],[],"[(ovarian cancer, TTL1), (breast cancer, TTL1)]","[[Ovarian Cancer, , , , ], [Ovarian Cancer, ov...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104762,NCT04767152,NCT04767152\nAcute Leukemia\nQuantitative MRI\...,"{\n ""tumors"": [\n {\n ""type"": ""Acute ...","{\n ""tumors"": [\n {\n ""type"": ""acute ...","[('NCT04767152', 'nct'), ('Acute Leukemia', 'T...",('acute lymphoblastic leukemia TTL2 stage null...,[[]],[],[],[],[],[],[],None,None
104763,NCT04115995,NCT04115995\nReconstructive Surgical Procedure...,"{\n ""tumors"": [\n {\n ""type"": ""Breast...","{\n ""tumors"": [\n {\n ""type"": ""breast...","[('NCT04115995', 'nct'), ('Reconstructive Surg...","('breast cancer TTL1 stage null null', [([('br...","[[{'Tumor Type': 'breast cancer', 'Tag': 'TTL1...","[{'Tumor Type': 'breast cancer', 'Tag': 'TTL1'}]",[],[],[],[],"[(breast cancer, TTL1)]","[[Breast Cancer, , , , ], [Breast Cancer, Inva...",[]
104764,NCT04412135,NCT04412135\nHematologic Cancer\nNone\nThe Poi...,"{\n ""tumors"": [\n {\n ""type"": ""Hemato...","{\n ""tumors"": [\n {\n ""type"": ""hemato...","[('NCT04412135', 'nct'), ('Hematologic Cancer'...","('hematologic cancer TTL1 stage null null', [(...",[[]],[],[],[],[],[],[],None,None
104765,NCT05490017,NCT05490017\nParoxysmal Nocturnal Hemoglobinur...,"{\n ""tumors"": []\n}","{\n ""tumors"": []\n}","[('NCT05490017', 'nct'), ('Paroxysmal Nocturna...","('', [])",[[]],[],[],[],[],[],[],None,None


In [64]:
df_palm.to_excel("./Output_with_new_M_File.xlsx", index=False)

In [34]:
import pandas as pd
df_palm=pd.read_excel(r"C:\Users\basem\OneDrive\Desktop\shakaib\Tumor module\TTL pkg\TTL_updated_Master_File_Dec_2024\Code_files\Output_with_new_M_File.xlsx")
df_palm.columns

Index(['NCTId', 'concatenated_text', 'output_PALM2', 'output_PALM2_updated',
       'NERs', 'basem_transformed_values', 'prod_over_palm', 'ttl_tags', 'met',
       'anat', 'other_tags', 'Tumor_Type_Missed_Tags',
       'prodigy_over_palm_refined', 'Post_Processing_results',
       'No_post_processing_equivalency'],
      dtype='object')

## Extracting the unique tumor types

In [35]:
import pandas as pd
import ast

# Inspect the first few entries before conversion
print("Original 'No_post_processing_equivalency' Column:")
print(df_palm['No_post_processing_equivalency'].head(10))

# Count NaN values
nan_count = df_palm['No_post_processing_equivalency'].isna().sum()
print(f"Number of NaN entries: {nan_count}")

# Identify malformed entries
def is_valid_list_format(s):
    if isinstance(s, str):
        s = s.strip()
        return s.startswith('[') and s.endswith(']')
    return False

malformed_entries = df_palm[~df_palm['No_post_processing_equivalency'].apply(is_valid_list_format) & 
                             df_palm['No_post_processing_equivalency'].notna()]

print(f"Number of malformed entries: {len(malformed_entries)}")
print("Malformed Entries:")
print(malformed_entries['No_post_processing_equivalency'].head(10))

# ----------------------------- Step 1: Define Safe Conversion Function ----------------------------- #
def safe_literal_eval(cell):
    """
    Safely evaluate a string containing a Python literal (e.g., list).
    If the cell is NaN or not a string, return an empty list.
    If the string cannot be parsed, return an empty list.
    """
    if isinstance(cell, str):
        try:
            # Attempt to parse the string to a Python list
            return ast.literal_eval(cell)
        except (ValueError, SyntaxError):
            print(f"Warning: Could not parse the cell content: {cell}. Assigning empty list.")
            return []
    elif isinstance(cell, list):
        # If it's already a list, return as is
        return cell
    else:
        # For NaN or other types, return an empty list
        return []

# ----------------------------- Step 2: Apply Safe Conversion ----------------------------- #
# Apply the safe conversion function to the column
df_palm['No_post_processing_equivalency'] = df_palm['No_post_processing_equivalency'].apply(safe_literal_eval)

# Verify the conversion
print("Converted 'No_post_processing_equivalency' Column:")
print(df_palm['No_post_processing_equivalency'].head(10))
print(type(df_palm['No_post_processing_equivalency'].iloc[0]))  # Should output <class 'list'>

# ----------------------------- Step 3: Explode the Lists ----------------------------- #
# Explode the 'No_post_processing_equivalency' column into separate rows
df_exploded = df_palm.explode('No_post_processing_equivalency')

# Reset the index for cleanliness
df_exploded.reset_index(drop=True, inplace=True)

# Display the first few rows of the exploded DataFrame
print("Exploded DataFrame:")
print(df_exploded.head(10))

# ----------------------------- Step 4: Clean the Data ----------------------------- #
# Remove leading and trailing whitespaces
df_exploded['No_post_processing_equivalency'] = df_exploded['No_post_processing_equivalency'].astype(str).str.strip()

# Replace 'nan' strings with empty strings
df_exploded['No_post_processing_equivalency'] = df_exploded['No_post_processing_equivalency'].replace('nan', '')

# Optionally, remove rows with empty 'No_post_processing_equivalency'
# df_exploded = df_exploded[df_exploded['No_post_processing_equivalency'] != '']

# Reset index again after removal
df_exploded.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print("Exploded and Cleaned DataFrame:")
print(df_exploded.head(10))

# ----------------------------- Step 5: Remove Duplicate Entries ----------------------------- #
# Remove duplicate NCTId-term pairs
df_exploded = df_exploded.drop_duplicates(subset=['NCTId', 'No_post_processing_equivalency'])

# Verify the removal
print(f"Number of rows after removing duplicates: {len(df_exploded)}")

# ----------------------------- Step 6: Group and Aggregate ----------------------------- #
# Group by 'No_post_processing_equivalency' and aggregate 'NCTId's into lists
df_unique = df_exploded.groupby('No_post_processing_equivalency')['NCTId'].apply(list).reset_index()

# Display the resulting DataFrame
print("Unique Terms with Associated NCTIds:")
print(df_unique.head(10))

Original 'No_post_processing_equivalency' Column:
0                                                   []
1                                                  NaN
2                                                   []
3                                                  NaN
4                                                   []
5                                                   []
6                                                  NaN
7                                                  NaN
8    ['colorectal cancer (includes colon and rectum...
9                                                   []
Name: No_post_processing_equivalency, dtype: object
Number of NaN entries: 39396
Number of malformed entries: 0
Malformed Entries:
Series([], Name: No_post_processing_equivalency, dtype: object)
Converted 'No_post_processing_equivalency' Column:
0                                                   []
1                                                   []
2                                           

In [37]:
df_exploded.to_excel("./Full_Output_With_Terms_Not_Found_Incl_Blanks.xlsx", index="False")

In [32]:
import pandas as pd
import ast

# Inspect the first few entries before conversion
print("Original 'No_post_processing_equivalency' Column:")
print(df_palm['No_post_processing_equivalency'].head(10))

# Count NaN values
nan_count = df_palm['No_post_processing_equivalency'].isna().sum()
print(f"Number of NaN entries: {nan_count}")

# Identify malformed entries
def is_valid_list_format(s):
    if isinstance(s, str):
        s = s.strip()
        return s.startswith('[') and s.endswith(']')
    return False

malformed_entries = df_palm[~df_palm['No_post_processing_equivalency'].apply(is_valid_list_format) & 
                             df_palm['No_post_processing_equivalency'].notna()]

print(f"Number of malformed entries: {len(malformed_entries)}")
print("Malformed Entries:")
print(malformed_entries['No_post_processing_equivalency'].head(10))

# ----------------------------- Step 1: Define Safe Conversion Function ----------------------------- #
def safe_literal_eval(cell):
    """
    Safely evaluate a string containing a Python literal (e.g., list).
    If the cell is NaN or not a string, return an empty list.
    If the string cannot be parsed, return an empty list.
    """
    if isinstance(cell, str):
        try:
            # Attempt to parse the string to a Python list
            return ast.literal_eval(cell)
        except (ValueError, SyntaxError):
            print(f"Warning: Could not parse the cell content: {cell}. Assigning empty list.")
            return []
    elif isinstance(cell, list):
        # If it's already a list, return as is
        return cell
    else:
        # For NaN or other types, return an empty list
        return []

# ----------------------------- Step 2: Apply Safe Conversion ----------------------------- #
# Apply the safe conversion function to the column
df_palm['No_post_processing_equivalency'] = df_palm['No_post_processing_equivalency'].apply(safe_literal_eval)

# Verify the conversion
print("Converted 'No_post_processing_equivalency' Column:")
print(df_palm['No_post_processing_equivalency'].head(10))
print(type(df_palm['No_post_processing_equivalency'].iloc[0]))  # Should output <class 'list'>

# ----------------------------- Step 3: Explode the Lists ----------------------------- #
# Explode the 'No_post_processing_equivalency' column into separate rows
df_exploded = df_palm.explode('No_post_processing_equivalency')

# Reset the index for cleanliness
df_exploded.reset_index(drop=True, inplace=True)

# Display the first few rows of the exploded DataFrame
print("Exploded DataFrame:")
print(df_exploded.head(10))

# ----------------------------- Step 4: Clean the Data ----------------------------- #
# Remove leading and trailing whitespaces
df_exploded['No_post_processing_equivalency'] = df_exploded['No_post_processing_equivalency'].astype(str).str.strip()

# Replace 'nan' strings with empty strings
df_exploded['No_post_processing_equivalency'] = df_exploded['No_post_processing_equivalency'].replace('nan', '')

# Optionally, remove rows with empty 'No_post_processing_equivalency'
df_exploded = df_exploded[df_exploded['No_post_processing_equivalency'] != '']

# Reset index again after removal
df_exploded.reset_index(drop=True, inplace=True)

# Display the cleaned DataFrame
print("Exploded and Cleaned DataFrame:")
print(df_exploded.head(10))

# ----------------------------- Step 5: Remove Duplicate Entries ----------------------------- #
# Remove duplicate NCTId-term pairs
df_exploded = df_exploded.drop_duplicates(subset=['NCTId', 'No_post_processing_equivalency'])

# Verify the removal
print(f"Number of rows after removing duplicates: {len(df_exploded)}")

# ----------------------------- Step 6: Group and Aggregate ----------------------------- #
# Group by 'No_post_processing_equivalency' and aggregate 'NCTId's into lists
df_unique = df_exploded.groupby('No_post_processing_equivalency')['NCTId'].apply(list).reset_index()

# Display the resulting DataFrame
print("Unique Terms with Associated NCTIds:")
print(df_unique.head(10))

# ----------------------------- Step 6.2: Remove Duplicate NCTIds Within Lists ----------------------------- #
def remove_duplicates(nctid_list):
    seen = set()
    unique_list = []
    for nctid in nctid_list:
        if nctid not in seen:
            unique_list.append(nctid)
            seen.add(nctid)
    return unique_list

df_unique['NCTId'] = df_unique['NCTId'].apply(remove_duplicates)

# Verify the cleaning
print("After Removing Duplicate NCTIds:")
print(df_unique.head(10))

# ----------------------------- Step 6.3: Sort NCTId Lists ----------------------------- #
df_unique['NCTId'] = df_unique['NCTId'].apply(lambda x: sorted(x))

# Verify the sorting
print("After Sorting NCTIds:")
print(df_unique.head(10))

# ----------------------------- Step 6.4: Rename Columns ----------------------------- #
# Rename columns for clarity
df_unique.rename(columns={
    'No_post_processing_equivalency': 'Term',
    'NCTId': 'Associated_NCTIds'
}, inplace=True)

# Display the updated DataFrame
print("Renamed Columns:")
print(df_unique.head(10))

# ----------------------------- Step 7: Save the Final DataFrame ----------------------------- #
# Save to XLSX
df_unique.to_excel('unique_terms_with_NCTIds.xlsx', index=False)
print("Data successfully saved to 'unique_terms_with_NCTIds.xlsx'.")


Original 'No_post_processing_equivalency' Column:
0                                                   []
1                                                  NaN
2                                                   []
3                                                  NaN
4                                                   []
5                                                   []
6                                                  NaN
7                                                  NaN
8    ['colorectal cancer (includes colon and rectum...
9                                                   []
Name: No_post_processing_equivalency, dtype: object
Number of NaN entries: 39396
Number of malformed entries: 0
Malformed Entries:
Series([], Name: No_post_processing_equivalency, dtype: object)
Converted 'No_post_processing_equivalency' Column:
0                                                   []
1                                                   []
2                                           